In [1]:
import pandas
import ipynb.fs.full.data_operations as do
import ipynb.fs.full.feed_reader as fr
import ipynb.fs.full.dict_maker as dictionary
import ipynb.fs.full.backtester as bt

In [2]:
class main:
    @staticmethod
    def insert_prev_recos_in_table():
        recos = pandas.read_csv('../../data/prev_recos.csv', sep=',', na_values=['None'])
        recos['reco_date'] = pandas.to_datetime(recos['reco_date']).dt.strftime('%Y-%m-%d')
        recos.replace('None', None, inplace=True)
        #print(recos.head)
        db = do.mysql_Database('analyst_feeds')
        db.insert_values_in_table('reco_repository', recos)

    @staticmethod
    def get_article_links(site_links: dict):
        db = do.mysql_Database('analyst_feeds')                                            #Initialize the MySQL DB

        for site, value in site_links.items():          
            for idx, sub_site in enumerate(value):
                url = dictionary.dict_to_json.get_value_from_file(site, sub_site)        #Get the url for this sub-site for this site
                #print(f'{site} - {sub_site} - {url}')
                feeder = fr.feed_reader.read_feed(site, url)                          #Get all the links for feed for this sub-site as a DataFrame
                
                if(not feeder.empty):                                #sometimes feeds can also be empty
                    feeder.insert(1, 'sub_site_name', sub_site)
                    print(f'Found {feeder.shape[0]} links. Now deduping...')
                    links_seen = db.search_values_in_table('feeds_seen', 'link_id', feeder['link_id'], ['site_name', 'sub_site_name'], [site, sub_site], ['link_id'])       #Get values which are already in the table
                    print(f'Found {links_seen.shape[0]} links which have been analysed before...')
                    links_to_add = feeder[~feeder['link_id'].isin(links_seen['link_id'])]                    #remove values already in the table
                    if(not links_to_add.empty):   #if there are links worth adding. Writing an empty df to the table with replace clause will cause it to delete all other values  
                        print(f'Inserting {links_to_add.shape[0]} new links to the table...')
                        db.insert_values_in_table('feeds_seen', links_to_add[['link_id', 'site_name', 'sub_site_name', 'link', 'link_date', 'title', 'link_summary']])                                         #Insert table values of links seen so as to not repeat
                else:
                    print(f'No feed found for {site} - {sub_site} today. ')
                
        db.extinguish_connection()
        print('DB connection closed. Exiting...')

    @staticmethod
    def one_time_db_cleanup(filename:str):
        df = pandas.read_csv(filename, sep=',',header=0)
        db = do.mysql_Database('analyst_feeds')                                            #Initialize the MySQL DB
        db.update_a_table_column(df)  

    @staticmethod
    def run_backtests_on_all_recos(analyst:str):
        db = do.mysql_Database('analyst_feeds')
        recos = db.search_values_in_table(tablename='reco_repository', test_field='analyst', match_values=pandas.Series(analyst), aux_fields=['site_name'], aux_field_values=['Livemint'])
        
        for idx, row in recos.iterrows():
            try:
                tester = bt.ticker_Data(ticker=row['ticker'], start_date=row['reco_date'].strftime('%Y-%m-%d'), in_price=row['reco_value'], tgt_price=row['tgt_value'], sl_price=row['sl_value'])
                if(tester.sanity_test(row)):
                    price_data = tester.get_price_data()
                    tester.simple_win_or_loss(price_data)
                    insert_df = pandas.DataFrame(
                            data={
                                'reco_id': [row['reco_id']],
                                'ticker': [row['ticker']],
                                'testable': [1],
                                'outcome': [json.dumps(tester.outcome)],
                                'result': [tester.outcome.get('result')],
                                'result_price': [tester.outcome_price],
                                'result_horizon': [tester.return_horizon],
                                'result_return': [tester.return_rate]
                            }
                        )
                    db.insert_values_in_table(tablename='reco_backtest_results', df=insert_df)
                else:
                    #print(f'Sanity test for row {idx} for ticker {tester.ticker} at start_date {tester.start_date} is False')
                    insert_df = pandas.DataFrame(
                            data={
                                'reco_id': [row['reco_id']],
                                'ticker': [row['ticker']],
                                'testable': [0]
                            }
                        )
                    db.insert_values_in_table(tablename='reco_backtest_results', df=insert_df)
            except Exception as e:
                print(f'Caught exception {e} in processing row {idx} with reco_id {row['reco_id']}.')
                insert_df = pandas.DataFrame(
                            data={
                                'reco_id': [row['reco_id']],
                                'ticker': [row['ticker']],
                                'testable': [1],
                                'result': ['failure'],
                            }
                        )
                db.insert_values_in_table(tablename='reco_backtest_results', df=insert_df)
            finally:
                continue

    #@staticmethod
    #def get_google_links
    